In [1]:
import pandas as pd
import numpy as np
from bson import ObjectId

import mysql.connector

In [2]:
def pullEIDataFromMySQL():
    connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
    )
    
    cursor = connection.cursor()
    
    query = "SELECT * FROM ei"
    cursor.execute(query)
    
    rows = cursor.fetchall()
    
    df = pd.DataFrame(rows, columns=['_id', 'FirstName', 'LastName', 'ServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'Longitude', 'Latitude'])
    df["Role"] = "ei"
    
    cursor.close()
    connection.close()
    
    return df

In [3]:
def pullChildDataFromMySQL():
    connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
    )
    
    cursor = connection.cursor()
    
    query = "SELECT * FROM child"
    cursor.execute(query)
    
    rows = cursor.fetchall()
    
    df = pd.DataFrame(rows, columns=['_id', 'FirstName', 'LastName', 'ServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'Longitude', 'Latitude'])
    df["Role"] = "child"
    
    cursor.close()
    connection.close()
    
    return df

In [4]:
dataEI = pullEIDataFromMySQL()
dataChild = pullChildDataFromMySQL()
allData = pd.concat([dataEI, dataChild], ignore_index=True)
print(allData)

                          _id FirstName    LastName  ServiceHours  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson           0.0   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton           1.0   
2    65d75c59e09e67e025e8cd32     Kassi      Knight           0.0   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor           0.0   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover           0.0   
..                        ...       ...         ...           ...   
204  65e6038938f826f29267202f     Judah     Overbay           1.0   
205  65e603f238f826f292672030    Bianca       Pogue           1.0   
206  65e6044938f826f292672031   Emorett     Woodall           1.0   
207  65e6049b38f826f292672032      Remi       Woody           0.5   
208  65ef076606e0a866af56d202  Madilynn      Phipps           1.0   

                      Address1           City State Zipcode      County  \
0     904 West Hillcrest Drive   Johnson City    TN   37604  Washington   
1        151 Valley V

In [10]:
connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
)
    
cursor = connection.cursor()

for index1, row1 in allData.iterrows():
    for index2, row2 in allData.iterrows():
        Role1 = row1["Role"]
        fromID1 = row1["_id"]
        Role2 = row2["Role"]
        fromID2 = row2["_id"]
        
        if fromID1 != fromID2:
            if Role1 == "child":
                if Role2 == "child":
                    query = ("SELECT COUNT(*) from locationdata "
                             f"WHERE child_id_1 = '{fromID1}' and child_id_2 = '{fromID2}' "
                             f"OR child_id_1 = '{fromID2}' and child_id_2 = '{fromID1}'")
                elif Role2 == "ei":
                    query = ("SELECT COUNT(*) from locationdata "
                             f"WHERE child_id_1 = '{fromID1}' and ei_id_1 = '{fromID2}' ")
                
            elif Role1 == "ei":
                if Role2 == "child":
                    query = ("SELECT COUNT(*) from locationdata " 
                             f"WHERE ei_id_1 = '{fromID1}' and child_id_1 = '{fromID2}'")
                if Role2 == "ei":
                    query = ("SELECT COUNT(*) from locationdata "
                             f"WHERE ei_id_1 = '{fromID1}' and ei_id_2 = '{fromID2}' " 
                             f"OR ei_id_1 = '{fromID2}' and ei_id_2 = '{fromID1}'")
            else:
                print("Role Error")
                query = ("")

            cursor.execute(query)
            rows = cursor.fetchall()
            if rows[0][0] != 1:
                print(f"From {row1['FirstName']} {row1['LastName']} to {row2['FirstName']} {row2['LastName']}: {rows[0][0]}")
                print("\n")
    
cursor.close()
connection.close()

In [8]:
connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
)
    
cursor = connection.cursor()

query = ("Select * from locationdata "
         f"WHERE child_id_1 = '65e5fd6538f826f292672024' or child_id_2 = '65e5fd6538f826f292672024'")
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    print(row)
    print(row[1])
    print(row[1] == "65e5fd6538f826f292672024")

    if row[1] == "65e5fd6538f826f292672024":
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            "65e5fe7438f826f292672025", row[2], row[3], row[4], row[5], row[6], row[7]))
        connection.commit()
        print("is inserted in the MySQL database")
        
    elif row[2] == "65e5fd6538f826f292672024":
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            row[1], "65e5fe7438f826f292672025", row[3], row[4], row[5], row[6], row[7]))
        connection.commit()
        print("is inserted in the MySQL database")
    else:
        print("Error")

        
cursor.close()
connection.close()

(1146, '65e5fd6538f826f292672024', None, '65d75a2ce09e67e025e8cd30', None, 0, 12.3468, 19.4323)
65e5fd6538f826f292672024
True
is inserted in the MySQL database
(1360, '65e5fd6538f826f292672024', None, '65d75bb0e09e67e025e8cd31', None, 0, 41.5838, 45.2369)
65e5fd6538f826f292672024
True
is inserted in the MySQL database
(1565, '65e5fd6538f826f292672024', None, '65d75c59e09e67e025e8cd32', None, 0, 36.3551, 40.8861)
65e5fd6538f826f292672024
True
is inserted in the MySQL database
(1769, '65e5fd6538f826f292672024', None, '65d75c87e09e67e025e8cd33', None, 0, 13.9953, 22.6969)
65e5fd6538f826f292672024
True
is inserted in the MySQL database
(1972, '65e5fd6538f826f292672024', None, '65d75cb9e09e67e025e8cd34', None, 0, 25.3009, 34.9181)
65e5fd6538f826f292672024
True
is inserted in the MySQL database
(2174, '65e5fd6538f826f292672024', None, '65d75ce1e09e67e025e8cd35', None, 0, 9.44552, 15.2209)
65e5fd6538f826f292672024
True
is inserted in the MySQL database
(2375, '65e5fd6538f826f292672024', None,